# Algebra of drawing functions

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

The topics discussed are:

- a drawing function as we have built so far in general takes `n*n` decisions of the form "is the current pixel an asterisk?"
- we could generalise this in a HOF:

In [5]:
def draw(n,m,p):
    s = ""
    i = 0
    while i < n:
        j = 0
        while j < m:
            if p(n,m,i,j):
                s = s + "*"
            else:
                s = s + " "
            j = j + 1
        s = s + "\n"
        i = i + 1
    return s

- this function accepts as arguments the size of the picture to draw, and a function `p`
    - by specifying different values of `p` (as lambda expressions if needed), we completely change the picture drawn
    - for example, if `p` always returns `True`, we simply fill the picture with asterisks
    - on the other hand, if `p` always returns `False`, we simply get a completely empty picture
    - when `j <= i` then we get a triangle
    - and so on...

In [6]:
print(draw(3,3,lambda n,m,i,j: True))
print(draw(3,3,lambda n,m,i,j: False))
print(draw(3,3,lambda n,m,i,j: j <= i))

***
***
***

   
   
   

*  
** 
***



- interestingly enough, we could now simply state that a function from four integers (width, height, row index, column index) into a boolean (pixel on/off) **is** a picture
    - we can simply pass such a function to `draw` and just see it drawn
- let us now consider a hollow square
- let us identify the borders: `i == 0`, `j == 0`, `i == n-1`, or `j == n-1`

In [8]:
print(draw(3,3,lambda n,m,i,j: i == 0))
print(draw(3,3,lambda n,m,i,j: j == 0))
print(draw(3,3,lambda n,m,i,j: i == n-1))
print(draw(3,3,lambda n,m,i,j: j == m-1))

***
   
   

*  
*  
*  

   
   
***

  *
  *
  *



- in the same spirit as the previous lectures, we always look for abstractions that capture our idea of composing concepts together
- given multiple functions such as `lambda n,m,i,j: i == 0`, `lambda n,m,i,j: j == 0`, etc., how do we compose them into a single function which represents the picture drawn by, for example, overlaying the pictures?
- we could do this explicitly (and thus quite verbosely), as a first approximation
    - notice that the hollow square draws pixels which belong to the upper row, lower row, left column, **or** right column
    - we can literally just invoke the various functions for the image parts and combine the results in a boolean `or`

In [10]:
upper_line = lambda n,m,i,j: i == 0
left_line = lambda n,m,i,j: j == 0
lower_line = lambda n,m,i,j: i == n-1
right_line = lambda n,m,i,j: j == m-1

hollow_square = lambda n,m,i,j: upper_line(n,m,i,j) or lower_line(n,m,i,j) or left_line(n,m,i,j) or right_line(n,m,i,j)
print(draw(3,3,hollow_square))

***
* *
***



- suppose now that we wanted to just draw the upper and lower rows
- we would have to define another function, and even figure out some reasonable name for it

In [11]:
upper_and_lower_rows = lambda n,m,i,j: upper_line(n,m,i,j) or lower_line(n,m,i,j)
print(draw(3,3,upper_and_lower_rows))

***
   
***



- of course we could now notice an emerging pattern: composing a picture as an `or` of multiple pictures
- we could therefore define this pattern explicitly as a higher order function which takes two pictures as parameters, invokes both of them, and combines the results with an `or`

In [12]:
def pic_or(p,q):
    return lambda n,m,i,j: p(n,m,i,j) or q(n,m,i,j)
print(draw(3,3,pic_or(upper_line, lower_line)))

***
   
***



- thanks to `pic_or`, which we call a _combinator_ (since it combines arbitrary pictures together), we can quickly draw different pictures

In [20]:
print(draw(3,3,pic_or(upper_line, left_line)))
print(draw(6,6,pic_or(lambda n,m,i,j: n-i-1 <= j, lambda n,m,i,j: j <= i)))

***
*  
*  

*    *
**  **
******
******
******
******



- given that `pic_or` is itself a picture, we can pass its result to `pic_or` itself in order to go on nesting

In [22]:
print(draw(5,5,pic_or(pic_or(upper_line, lower_line), pic_or(left_line, right_line))))

*****
*   *
*   *
*   *
*****



- of course there must be many other combinators possible
- in order to discover these combinators, we could: 
    - perform a long exploration based on trial and error, and look for new and interesting ways to combine pictures, or
    - we could notice that pictures return booleans, and as such given multiple pictures the ways we can combine their output per pixel define the ways we can combine the pictures themselves
        - booleans can be combined with `and`, `or`, and `not`
        - `and` of two pictures produces their intersection, that is only the pixels which are lit in both pictures
        - `not` inverts the picture: the lit pixels become unlit, and vice-versa

In [24]:
def pic_and(p,q):
    return lambda n,m,i,j: p(n,m,i,j) and q(n,m,i,j)
print(draw(3,3,pic_and(upper_line, left_line)))

def pic_not(p):
    return lambda n,m,i,j: not p(n,m,i,j)
print(draw(3,3,pic_not(left_line)))

*  
   
   

 **
 **
 **



- we have performed transformations on the output of the picture functions
- we can also perform transformations on the parameters
    - one such transformation is flipping, horizontally and vertically
        - for example, we could invoke the picture function with `n-1` instead of `0`, `n-2` instead of `1`, etc.

In [29]:
def flip_hor(p):
    return lambda n,m,i,j: p(n,m,i,m-1-j)
def flip_ver(p):
    return lambda n,m,i,j: p(n,m,n-1-i,j)
upper_right_triangle = lambda n,m,i,j: i <= j
upper_left_triangle  = flip_hor(upper_right_triangle)
lower_right_triangle = flip_ver(upper_right_triangle)
lower_left_triangle  = flip_ver(upper_left_triangle)
print(draw(3,3,upper_left_triangle))
print(draw(3,3,upper_right_triangle))
print(draw(3,3,lower_left_triangle))
print(draw(3,3,lower_right_triangle))

***
** 
*  

***
 **
  *

*  
** 
***

  *
 **
***



- we could also notice that a pyramid is no more than the intersection of the two lower triangles, but now other pyramids which would have been much harder to draw also become trivial

In [35]:
upper_pyramid = pic_and(upper_left_triangle, upper_right_triangle)
lower_pyramid = pic_and(lower_left_triangle, lower_right_triangle)
left_pyramid = pic_and(lower_left_triangle, upper_left_triangle)
right_pyramid = pic_and(lower_right_triangle, upper_right_triangle)
print(draw(5,5,lower_pyramid))
print(draw(5,5,left_pyramid))


butterfly = pic_or(left_pyramid, right_pyramid)
print(draw(5,5,butterfly))

     
     
  *  
 *** 
*****

*    
**   
***  
**   
*    

*   *
** **
*****
** **
*   *



- as a final touch, let us create a circle by using the well-known Pythagoras' formula

In [39]:
circle = lambda n,m,i,j: (i-n//2)*(i-n//2)+(j-n//2)*(j-n//2) <= n*n//4
print(draw(5,5,circle))

 *** 
*****
*****
*****
 *** 

     *****     
   *********   
  **********   
 **********    
 *********     
*********      
********       
*******        
********       
*********      
 *********     
 **********    
  **********   
   *********   
     *****     



- let us conclude with a classic

In [45]:
print(draw(15,15,pic_and(circle, pic_not(pic_or(lambda n,m,i,j: i == n//4 and j == m//2, right_pyramid)))))

     *****     
   *********   
  **********   
 ****** ***    
 *********     
*********      
********       
*******        
********       
*********      
 *********     
 **********    
  **********   
   *********   
     *****     

